## Download the files
These include the train test images as well the csv indexing them

In [0]:
!wget -q https://s3.eu-central-1.wasabisys.com/aicrowd-practice-challenges/public/foodc/v0.1/train_images.zip
!wget -q https://s3.eu-central-1.wasabisys.com/aicrowd-practice-challenges/public/foodc/v0.1/test_images.zip
!wget -q https://s3.eu-central-1.wasabisys.com/aicrowd-practice-challenges/public/foodc/v0.1/train.csv
!wget -q https://s3.eu-central-1.wasabisys.com/aicrowd-practice-challenges/public/foodc/v0.1/test.csv


We create directories and unzip the images

In [0]:
!mkdir data
!mkdir data/test
!mkdir data/train
!unzip train_images -d data/train
!unzip test_images -d data/test

Streaming output truncated to the last 5000 lines.
  inflating: data/train/train_images/145896eed0.jpg  
  inflating: data/train/train_images/c468193dfd.jpg  
  inflating: data/train/train_images/7c6dccda37.jpg  
  inflating: data/train/train_images/c03811cd27.jpg  
  inflating: data/train/train_images/f04e23613b.jpg  
  inflating: data/train/train_images/d1cbbc1134.jpg  
  inflating: data/train/train_images/d762ff4fd3.jpg  
  inflating: data/train/train_images/1df8826db5.jpg  
  inflating: data/train/train_images/4f05c60d04.jpg  
  inflating: data/train/train_images/1b33d95837.jpg  
  inflating: data/train/train_images/fa3e225fe7.jpg  
  inflating: data/train/train_images/7bcd015a19.jpg  
  inflating: data/train/train_images/a413778c2b.jpg  
  inflating: data/train/train_images/4945972217.jpg  
  inflating: data/train/train_images/5245cffff8.jpg  
  inflating: data/train/train_images/6012b508f4.jpg  
  inflating: data/train/train_images/8bf820b404.jpg  
  inflating: data/train/train_i

In [0]:
!git clone https://github.com/ufoym/imbalanced-dataset-sampler.git
!rm -rf imbalanced-dataset-sampler
!cd imbalanced-dataset-sampler
!python imbalanced-dataset-sampler/setup.py install
!pip install pywick
!pip install efficientnet_pytorch

  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.6.3-cp36-none-any.whl size=12422 sha256=5778b937cd926ff1cb316c4e0196c4c905d28a10ab5f99b87b1d4f23c8676ae6
  Stored in directory: /root/.cache/pip/wheels/42/1e/a9/2a578ba9ad04e776e80bf0f70d8a7f4c29ec0718b92d8f6ccd
Successfully built efficientnet-pytorch


## Import necessary packages

In [0]:
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader, Dataset
import torchvision
from torchvision import models
import torch.optim as optim
import pandas as pd
import numpy as np
import cv2
import os
from sklearn import preprocessing
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Loading Data
In pytorch we can directly load our files into torchvision(the library which creates the object) or create a custom class to load data. The class must have `__init__` , `__len__` and `__getitem__` functions. We create a custom dataloader to suit our needs. More info on custom loaders can be read [here](https://pytorch.org/tutorials/beginner/data_loading_tutorial.html)

In [0]:
class FoodData(Dataset):
    def __init__(self,data_list,data_dir = './',transform=None,train=True):
        super().__init__()
        self.data_list = data_list
        self.data_dir = data_dir
        self.transform = transform
        self.train = train
    
    def __len__(self):
        return self.data_list.shape[0]
    
    def __getitem__(self,item):
        if self.train:
          img_name,label = self.data_list.iloc[item]
        else:
          img_name = self.data_list.iloc[item]['ImageId']
        img_path = os.path.join(self.data_dir,img_name)
        img = cv2.imread(img_path,1)
        img = cv2.resize(img,(128,128))

        if self.transform is not None:
            img = self.transform(img)
        if self.train:
          return img, torch.tensor(label)
          # return {
          #     'gt' : img,
          #     'label' : torch.tensor(label)

          # }
        else:
          return img
          # return {
              # 'gt':img
          # }
        

We first convert the data labels into encodings using Label Encoders. This basically converts labels into number encodings. This is an important step as without it we cannot train our network

In [0]:
train = pd.read_csv('train.csv')
le = preprocessing.LabelEncoder()
targets = le.fit_transform(train['ClassName'])
ntrain = train
ntrain['ClassName'] = targets

We load our train data and some necessary augementations like converting to PIL image, converting to tensors and normalizing them across channels. We can add more augementations such as `Random Flip`, `Random Rotation`, etc more on which can be found [here](https://pytorch.org/docs/stable/torchvision/transforms.html)

In [0]:
transforms_train = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomRotation(30),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                            std=[0.229, 0.224, 0.225])
])

train_path = 'data/train/train_images'
train_data = FoodData(data_list= ntrain,data_dir = train_path,transform = transforms_train)

## Split Data into Train and Validation
Now we want to see how well our model is performing, but we dont have the test data labels with us to check. What do we do ? So we split our dataset into train and validation. The idea is that we test our classifier on validation set in order to get an idea of how well our classifier works. This way we can also ensure that we dont [overfit](https://machinelearningmastery.com/overfitting-and-underfitting-with-machine-learning-algorithms/) on the train dataset. There are many ways to do validation like [k-fold](https://machinelearningmastery.com/k-fold-cross-validation/),[leave one out](https://en.wikipedia.org/wiki/Cross-validation_(statistics), etc  

We also make `dataloaders` which basically create minibatches of dataset which are used in each epoch

In [0]:
batch = 32

In [0]:
valid_size = 0.1
num = train_data.__len__()
random_seed = 42
# Dividing the indices for train and cross validation
indices = list(range(num))
np.random.seed(random_seed)
np.random.shuffle(indices)
split = int(np.floor(valid_size*num))
train_idx,valid_idx = indices[split:], indices[:split]

#Create Samplers
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)



In [0]:
train_loader = DataLoader(train_data, batch_size = batch, sampler = train_sampler)
valid_loader = DataLoader(train_data, batch_size = batch, sampler = valid_sampler)

In [0]:
transforms_test = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                            std=[0.229, 0.224, 0.225])
])

test_path = 'data/test/test_images'
test = pd.read_csv('test.csv')
test_data = FoodData(data_list= test,data_dir = test_path,transform = transforms_test,train=False)

test_loader = DataLoader(test_data, batch_size=batch, shuffle=False)


Here we check if we have a GPU or not. If we have we just need to shift our data and model to GPU for faster computations.

In [0]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Assuming that we are on a CUDA machine, this should print a CUDA device:

print(device)
    

cuda:0


In [0]:
class Network(nn.Module):
  def __init__(self, model1, model2):
    super(Network, self).__init__()
    self.model1 = model1
    self.model2 = model2

  def forward(self, x):
    out1 = self.model1(x)
    out2 = self.model2(x)

    return (out1 + out2)/2

In [0]:
criterion = nn.CrossEntropyLoss()                       # <--- Cross Entropy Loss    

In [0]:
from efficientnet_pytorch import EfficientNet
model_1 = EfficientNet.from_pretrained('efficientnet-b7', num_classes=61)

Loaded pretrained weights for efficientnet-b7


In [0]:
model_2 = models.densenet169(pretrained=True, progress=True)
model_2.classifier = nn.Linear(1664, 61)

In [0]:
model = Network(model_1, model_2)
model = model.cuda()
print(model)

Network(
  (model1): EfficientNet(
    (_conv_stem): Conv2dStaticSamePadding(
      3, 64, kernel_size=(3, 3), stride=(2, 2), bias=False
      (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
    )
    (_bn0): BatchNorm2d(64, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
    (_blocks): ModuleList(
      (0): MBConvBlock(
        (_depthwise_conv): Conv2dStaticSamePadding(
          64, 64, kernel_size=(3, 3), stride=[1, 1], groups=64, bias=False
          (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
        )
        (_bn1): BatchNorm2d(64, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
        (_se_reduce): Conv2dStaticSamePadding(
          64, 16, kernel_size=(1, 1), stride=(1, 1)
          (static_padding): Identity()
        )
        (_se_expand): Conv2dStaticSamePadding(
          16, 64, kernel_size=(1, 1), stride=(1, 1)
          (static_padding): Identity()
        )
        (_pro

In [0]:
optimizer = optim.Adam(model.parameters(), lr=0.007)

In [0]:
def train(trainloader, model, criterion, optimizer):

  model.train()

  losses = 0
  total = 0
  correct = 0

  for batch_idx, (inputs, targets) in enumerate(trainloader):
    inputs, targets = inputs.cuda(), targets.cuda()
    
    outputs = model(inputs)
    if batch_idx % 10 == 1:
      print(batch_idx, losses, (100*correct/total).cpu().numpy())
    
    loss =  criterion(outputs, targets)

    _, predicted = torch.max(outputs.data, 1)
    
    total += targets.size(0)
    correct += (predicted == targets).sum()
    losses += loss.item()*inputs.size(0)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
  return (losses / total, 100.0 * correct / total)

def validate(validloader, model, criterion):
  model.eval()

  losses = 0
  total = 0
  correct = 0

  with torch.no_grad():
    for batch_idx, (inputs, targets) in enumerate(validloader):
      inputs, targets = inputs.cuda(), targets.cuda()
      
      outputs = model(inputs)
      loss =  criterion(outputs, targets)

      _, predicted = torch.max(outputs.data, 1)
      
      total += targets.size(0)
      correct += (predicted == targets).sum()
      losses += loss.item()*inputs.size(0)
    

  return (losses / total, 100.0 * correct / total)

NUM_EPOCHS = 10
def run_model(trainloader, validloader, model, criterion, optimizer):
  import time
  train_losses = []
  train_accuracy = []

  valid_losses = []
  valid_accuracy = []

  for epoch in range(NUM_EPOCHS):
    st = time.time()
    tlosses, tacc = 0,0
    
    tlosses, tacc = train(trainloader, model, criterion, optimizer)
    # scheduler.step()
    vlosses, vacc = validate(validloader, model, criterion)
    torch.save(model.state_dict(), 'drive/My Drive/aicrowd_blitz/food-ensemble.pth')

    print('Epoch: {}:: train_loss: {}, train_accuracy: {} \n valid_loss: {}, valid_accuracy: {}'.format(
        epoch, tlosses, tacc,
        vlosses, vacc
    ))
    train_losses.append(tlosses)
    train_accuracy.append(tacc)
    valid_losses.append(vlosses)
    valid_accuracy.append(vacc)
    
    en = time.time()
    print("epoch took:", en - st)

  return {
      'train' : (train_losses, train_accuracy),
      'val'  : (valid_losses, valid_accuracy),
  }

result = run_model(train_loader, valid_loader, model, criterion, optimizer)

1 16.67560386657715 84
11 254.26177978515625 76
21 504.4364194869995 75
31 720.3024520874023 77
41 945.7427530288696 77
51 1191.2278394699097 77
61 1384.6848936080933 77
71 1615.293565750122 77
81 1840.533555984497 78
91 2060.023241043091 77
101 2270.489288330078 78
111 2486.6472568511963 78
121 2697.164406776428 78
131 2899.374975204468 78
141 3119.663682937622 78
151 3330.2267627716064 78
161 3576.0008182525635 78
171 3782.3716192245483 78
181 3988.2672548294067 78
191 4250.754904747009 78
201 4478.363389015198 77
211 4679.274739265442 77
221 4903.979518890381 78
231 5134.285893440247 77
241 5367.065819740295 77
251 5594.098927497864 77
261 5789.794036865234 77
Epoch: 0:: train_loss: 0.6950701065277147, train_accuracy: 77.82147216796875 
 valid_loss: 0.8442510821788608, valid_accuracy: 73.17596435546875
epoch took: 561.191258430481


Following are the steps to train the classifier:


*   Train both efficient net and densenet models on the food dataset separately
*   Use the trained models from previous step to make an ensemble model using average fusion
*   Train the ensemble model again on the dataset
*   All the models need to be trained on different scales to make them more robust, so they were first trained on (128, 128) images and then for some epoches on (256, 256) and then again (128, 128) images
*   For the first few epochs when training the trained model on (256, 256) images, train only the fc layer (by freezing the rest of the model) and then unfreeze the rest of the model and train. Do the same when training on (128, 128) after training on (256, 256).



## Predict on test set
Time for the moment of truth! Predict on test set and time to make the submission.

In [0]:
torch.save(model.state_dict(), 'drive/My Drive/aicrowd_blitz/food-ensemble.pth')

In [0]:
# model.load_state_dict(torch.load('drive/My Drive/aicrowd_blitz/food-ensemble.pth'))
model.eval()

preds = []
with torch.no_grad():
    for images in test_loader:
        data = images.to(device)
        outputs = model(data)
        _, predicted = torch.max(outputs.data, 1)
        pr = predicted.detach().cpu().numpy()
        for i in pr:
          preds.append(i)


## Save it in correct format

In [0]:
# Create Submission file        
df = pd.DataFrame(le.inverse_transform(preds),columns=['ClassName'])
df.to_csv('submission-ensemble.csv',index=False)

## To download the generated in collab csv run the below command

In [0]:
from google.colab import files
files.download('submission-ensemble.csv') 

### Go to [platform](https://www.aicrowd.com/challenges/aicrowd-blitz-may-2020/problems/foodc). Participate in the challenge and submit the submission.csv.